In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [96]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [97]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.iloc[:,3:12]
Y_columns = dataset.iloc[:,-1]
print(X_columns)
print(Y_columns)

      CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  
0             1              

In [100]:
# We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X_columns["Gender"] = X_column_transformer.fit_transform(X_columns["Gender"] )#gender
X_columns

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember
0,619,France,0,42,2,0.00,1,1,1
1,608,Spain,0,41,1,83807.86,1,0,1
2,502,France,0,42,8,159660.80,3,1,0
3,699,France,0,39,1,0.00,2,0,0
4,850,Spain,0,43,2,125510.82,1,1,1
...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0
9996,516,France,1,35,10,57369.61,1,1,1
9997,709,France,0,36,7,0.00,1,0,1
9998,772,Germany,1,42,3,75075.31,2,1,0


In [102]:
# Lets Encode geography now
X_columns["Geography"] = X_column_transformer.fit_transform(X_columns["Geography"])
X_columns

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember
0,619,0,0,42,2,0.00,1,1,1
1,608,2,0,41,1,83807.86,1,0,1
2,502,0,0,42,8,159660.80,3,1,0
3,699,0,0,39,1,0.00,2,0,0
4,850,2,0,43,2,125510.82,1,1,1
...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0
9996,516,0,1,35,10,57369.61,1,1,1
9997,709,0,0,36,7,0.00,1,0,1
9998,772,1,1,42,3,75075.31,2,1,0


We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [24]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# pipeline = Pipeline(
#     [
#         ('Categorizer', ColumnTransformer(
#             [
#                 ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
#                 ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
#             ], 
#             remainder = 'passthrough', n_jobs = 1)),
#         ('Normalizer', StandardScaler())
#     ]
# )

In [104]:
#Standardize the features
from sklearn.preprocessing import StandardScaler
Scale=StandardScaler()
X_columns = Scale.fit_transform(X_columns)

In [105]:
X_columns

array([[-0.32622142, -0.90188624, -1.09598752, ..., -0.91158349,
         0.64609167,  0.97024255],
       [-0.44003595,  1.51506738, -1.09598752, ..., -0.91158349,
        -1.54776799,  0.97024255],
       [-1.53679418, -0.90188624, -1.09598752, ...,  2.52705662,
         0.64609167, -1.03067011],
       ...,
       [ 0.60498839, -0.90188624, -1.09598752, ..., -0.91158349,
        -1.54776799,  0.97024255],
       [ 1.25683526,  0.30659057,  0.91241915, ...,  0.80773656,
         0.64609167, -1.03067011],
       [ 1.46377078, -0.90188624, -1.09598752, ..., -0.91158349,
         0.64609167, -1.03067011]])

In [106]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_columns, Y_columns, test_size = 0.2, random_state = 0)

In [107]:
X_train.shape

(8000, 9)

In [108]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [109]:
#Initialize ANN
classifier = Sequential()

In [110]:
#Add input layer and hidden layer
classifier.add(Dense(units=6, activation = 'relu', input_shape = (X_train.shape[1], )))
# classifier.add(Dropout(rate = 0.1))

In [111]:
#Add second layer
classifier.add(Dense(units=6, activation = 'relu'))
# classifier.add(Dropout(rate = 0.1))

In [112]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [113]:
#Let us take a look at our network
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 6)                 60        
                                                                 
 dense_38 (Dense)            (None, 6)                 42        
                                                                 
 dense_39 (Dense)            (None, 1)                 7         
                                                                 
Total params: 109 (436.00 Byte)
Trainable params: 109 (436.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [114]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [115]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, validation_split = 0.1, batch_size = 32, epochs = 200)

Epoch 1/200
225/225 [==============================] - 4s 5ms/step - loss: 0.5180 - accuracy: 0.7921 - val_loss: 0.4733 - val_accuracy: 0.7950
Epoch 2/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4747 - accuracy: 0.7967 - val_loss: 0.4516 - val_accuracy: 0.7912
Epoch 3/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4523 - accuracy: 0.8003 - val_loss: 0.4377 - val_accuracy: 0.7987
Epoch 4/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4383 - accuracy: 0.8028 - val_loss: 0.4261 - val_accuracy: 0.7962
Epoch 5/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4280 - accuracy: 0.8056 - val_loss: 0.4182 - val_accuracy: 0.7975
Epoch 6/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4180 - accuracy: 0.8092 - val_loss: 0.4082 - val_accuracy: 0.8050
Epoch 7/200
225/225 [==============================] - 1s 3ms/step - loss: 0.4085 - accuracy: 0.8121 - val_loss: 0.3993 - val_accuracy: 0.8012

Epoch 58/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3452 - accuracy: 0.8590 - val_loss: 0.3335 - val_accuracy: 0.8687
Epoch 59/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3453 - accuracy: 0.8587 - val_loss: 0.3324 - val_accuracy: 0.8687
Epoch 60/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3454 - accuracy: 0.8589 - val_loss: 0.3330 - val_accuracy: 0.8662
Epoch 61/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3452 - accuracy: 0.8590 - val_loss: 0.3325 - val_accuracy: 0.8687
Epoch 62/200
225/225 [==============================] - 1s 5ms/step - loss: 0.3451 - accuracy: 0.8586 - val_loss: 0.3332 - val_accuracy: 0.8687
Epoch 63/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3448 - accuracy: 0.8594 - val_loss: 0.3333 - val_accuracy: 0.8675
Epoch 64/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3448 - accuracy: 0.8593 - val_loss: 0.3325 - val_accuracy:

Epoch 115/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8603 - val_loss: 0.3272 - val_accuracy: 0.8725
Epoch 116/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3392 - accuracy: 0.8606 - val_loss: 0.3273 - val_accuracy: 0.8687
Epoch 117/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3393 - accuracy: 0.8617 - val_loss: 0.3276 - val_accuracy: 0.8675
Epoch 118/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3395 - accuracy: 0.8618 - val_loss: 0.3273 - val_accuracy: 0.8712
Epoch 119/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3393 - accuracy: 0.8603 - val_loss: 0.3274 - val_accuracy: 0.8712
Epoch 120/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3396 - accuracy: 0.8612 - val_loss: 0.3272 - val_accuracy: 0.8675
Epoch 121/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3390 - accuracy: 0.8611 - val_loss: 0.3278 - val_ac

225/225 [==============================] - 1s 5ms/step - loss: 0.3374 - accuracy: 0.8611 - val_loss: 0.3261 - val_accuracy: 0.8700
Epoch 172/200
225/225 [==============================] - 1s 5ms/step - loss: 0.3374 - accuracy: 0.8621 - val_loss: 0.3244 - val_accuracy: 0.8687
Epoch 173/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3372 - accuracy: 0.8629 - val_loss: 0.3250 - val_accuracy: 0.8700
Epoch 174/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3370 - accuracy: 0.8619 - val_loss: 0.3244 - val_accuracy: 0.8712
Epoch 175/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3373 - accuracy: 0.8626 - val_loss: 0.3251 - val_accuracy: 0.8675
Epoch 176/200
225/225 [==============================] - 1s 3ms/step - loss: 0.3372 - accuracy: 0.8631 - val_loss: 0.3255 - val_accuracy: 0.8675
Epoch 177/200
225/225 [==============================] - 1s 4ms/step - loss: 0.3374 - accuracy: 0.8622 - val_loss: 0.3252 - val_accuracy: 0.8687

In [116]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 0s 3ms/step
[[0.2938525 ]
 [0.30291677]
 [0.14517151]
 [0.04757197]
 [0.11174773]]


In [117]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [118]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1523   72]
 [ 205  200]]


In [119]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.15 % of data was classified correctly


In [120]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8615